In [1]:
from typing import Iterator, List, Dict
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import re
import os
import glob
import json
from collections import defaultdict
import itertools
import random
from tqdm import tqdm
from sklearn.metrics import f1_score

from allennlp.data import Instance
from allennlp.data.fields import TextField, MetadataField, ListField, ArrayField, LabelField, MultiLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.token_indexers.wordpiece_indexer import PretrainedBertIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.tokenizers.word_splitter import BertBasicWordSplitter
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset import Batch
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.attention.bilinear_attention import BilinearAttention
from allennlp.nn import util
from allennlp.training.metrics import CategoricalAccuracy, BooleanAccuracy
from allennlp.training.metrics.metric import Metric
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.modules.token_embedders.bert_token_embedder import BertEmbedder, PretrainedBertEmbedder

import warnings
warnings.filterwarnings("ignore")

In [2]:
EXP = 'st_baseline2-5'
CUDA = 2
# correct loss function
# add frame_change indicator
# trainable GRU init state
# -2 layer hidden

In [3]:
bert_model = 'bert-base-cased'
SLOT_ACTS = dict(INFORM=0, CONFIRM=1, OFFER=2, REQUEST=3)
GENERAL_ACTS = dict(NOTIFY_SUCCESS=0, NOTIFY_FAILURE=1, REQ_MORE=2, GOODBYE=3, INFORM_COUNT=4)
class DialogueDatasetReader(DatasetReader):
    def __init__(self,
                 bert_model: str = bert_model,
                 token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        do_lowercase = 'uncased' in bert_model
        self.token_indexers = token_indexers or {
            "tokens": PretrainedBertIndexer(bert_model, do_lowercase=do_lowercase, use_starting_offsets=True)}
        self.tokenizer = BertBasicWordSplitter(do_lower_case=do_lowercase)
        self.intent_cache = {}
        self.slot_cache = {}


    def text_to_instance(self, dialogue_id: str,
                         turn_id: int,
                         user_utterance: str,
                         service: str,
                         general_encoding: np.ndarray,
                         intent_encoding: np.ndarray,
                         slot_encoding: np.ndarray,
                         active_intent: str = None,
                         requested_slots: List[str] = None) -> Instance:
        meta_field = MetadataField(dict(dialogue_id=dialogue_id, turn_id=turn_id, service=service))
        user_utterance_field = TextField(self.tokenizer.split_words(user_utterance), self.token_indexers)

        intent_field = ListField(
            [TextField(self.tokenizer.split_words(intent['description']), self.token_indexers)
             for intent in self.intent_schema[service]])
        slot_field = ListField(
            [TextField(self.tokenizer.split_words(slot['description']), self.token_indexers)
             for slot in self.slot_schema[service]])
        
        fields = {"meta": meta_field,
                  "user_utterance": user_utterance_field,
                  "intent_descriptions": intent_field,
                  "slot_descriptions": slot_field,
                  "general_acts": ArrayField(general_encoding),
                  "intent_acts": ArrayField(intent_encoding),
                  "slot_acts": ArrayField(slot_encoding)}

        if active_intent:
            fields['intent_label'] = ArrayField(np.array(self.intent_to_index[service][active_intent]))
            fields['slot_label'] = ArrayField(
                np.array([int(slot['name'] in requested_slots) for slot in self.slot_schema[service]]),
                padding_value=-1
            )

        return Instance(fields)


    def _read(self, file_path: str) -> Iterator[Instance]:
        
        with open(os.path.join(file_path, 'schema.json'), 'r') as f:
            schema = json.load(f)
        self.intent_schema = {x['service_name']: x['intents'] for x in schema}
        self.slot_schema = {x['service_name']: x['slots'] for x in schema}
        self.index_to_intent = {
            service: ['NONE', 'UNCHANGED'] + [intent['name'] for intent in intents] 
            for service, intents in self.intent_schema.items()
        }
        self.intent_to_index = {
            service: {intent: i for i, intent in enumerate(intents)}
            for service, intents in self.index_to_intent.items()
        }
        self.index_to_slot = {
            service: [slot['name'] for slot in slots] 
            for service, slots in self.slot_schema.items()
        }
        self.slot_to_index = {
            service: {slot: i for i, slot in enumerate(slots)}
            for service, slots in self.index_to_slot.items()
        }
        for file in sorted(glob.glob(os.path.join(file_path, 'dialogues*.json'))):
            with open(file, 'r') as f:
                ds = json.load(f)
            for dialog in ds:
                dialogue_id = dialog['dialogue_id']
                turn_id = 0
                last_service = None
                last_intent = None
                act_dict = {}
                intent_encoding = None
                for turn in dialog['turns']:
                    if turn['speaker'] == 'SYSTEM':  # system action encoding
                        for frame in turn['frames']:  # only one frame
                            service = frame['service']
                            general_encoding = np.zeros(len(GENERAL_ACTS) + 1)
                            intent_encoding = np.zeros(len(self.intent_schema[service]))
                            slot_encoding = np.zeros((len(self.slot_schema[service]), len(SLOT_ACTS)))
                            for action in frame['actions']:
                                if action['act'] == 'OFFER_INTENT':
                                    for offered_intent in action['values']:
                                        i = self.intent_to_index[service][offered_intent] - 2  # no NONE here
                                        intent_encoding[i] = 1
                                elif action['act'] in SLOT_ACTS:
                                    i = self.slot_to_index[service][action['slot']]
                                    j = SLOT_ACTS[action['act']]
                                    slot_encoding[i, j] = 1
                                else:
                                    i = GENERAL_ACTS[action['act']]
                                    general_encoding[i] = 1
                                    if action['act'] == 'INFORM_COUNT':
                                        general_encoding[-1] = int(action['values'][0])
                            act_dict[service] = (general_encoding, intent_encoding, slot_encoding)
                        turn_id += 1
                        continue
                    if len(turn['frames']) > 1:  # sort
                        sorted_frames = []
                        for frame in turn['frames']:
                            if frame['service'] == last_service:
                                sorted_frames = [frame] + sorted_frames
                            else:
                                sorted_frames.append(frame)
                        turn['frames'] = sorted_frames
                    for frame in turn['frames']:
                        user_utterance = turn['utterance']
                        service = frame['service']
                        general_encoding, intent_encoding, slot_encoding = act_dict.get(
                            service, 
                            (np.zeros(len(GENERAL_ACTS) + 1),
                             np.zeros(len(self.intent_schema[service])),
                             np.zeros((len(self.slot_schema[service]), len(SLOT_ACTS))))
                        )
                        if 'active_intent' in frame['state']:
                            if last_intent == frame['state']['active_intent']:
                                active_intent = 'UNCHANGED'
                            else:
                                active_intent = frame['state']['active_intent']
                                last_intent = active_intent
                            requested_slots = frame['state']['requested_slots']
                        else:
                            active_intent = None
                            requested_slots = None
                        new_service = 1 if service != last_service else 0
                        general_encoding = np.concatenate([general_encoding, [new_service]])
                        yield self.text_to_instance(dialogue_id, turn_id, user_utterance, service,
                                                    general_encoding, intent_encoding, slot_encoding,
                                                    active_intent, requested_slots)
                        last_service = service

In [4]:
class DialogueIterator(object):
    """
    a custom iterator providing each dialogue as a batch
    """
    def __init__(self):
        self._epochs = defaultdict(int)
        self._num_batches_cache = {}
        self._cache: Dict[int, List[TensorDict]] = defaultdict(list)
    
    
    def __call__(self, instances,
                 num_epochs: int = None,
                 shuffle: bool = True):
        key = id(instances)
        starting_epoch = self._epochs[key]

        if num_epochs is None:
            epochs: Iterable[int] = itertools.count(starting_epoch)
        else:
            epochs = range(starting_epoch, starting_epoch + num_epochs)

        for epoch in epochs:
            if key not in self._cache:
                self._cache[key] = self._create_tensor_dicts(instances)
            tensor_dicts = self._cache[key]
            if shuffle:
                random.shuffle(tensor_dicts)
            for tensor_dict in tensor_dicts:
                yield tensor_dict
            self._epochs[key] = epoch + 1
            
        
    def _create_tensor_dicts(self, instances):
        tensor_dicts = []
        dialog_dict = defaultdict(list)
        for instance in instances:
            dialogue_id = instance.fields['meta'].metadata['dialogue_id']
            dialog_dict[dialogue_id].append(instance)
        for dialog in dialog_dict.values():
            batch = Batch(dialog)
            if self.vocab is not None:
                batch.index_instances(self.vocab)
            padding_lengths = batch.get_padding_lengths()
            tensor_dict = batch.as_tensor_dict(padding_lengths)
            tensor_dicts.append(tensor_dict)
        return tensor_dicts

    
    def get_num_batches(self, instances) -> int:
        key = id(instances)
        if key not in self._num_batches_cache:
            self._num_batches_cache[key] = len(
                set(instance.fields['meta'].metadata['dialogue_id'] for instance in instances))
        return self._num_batches_cache[key]
        
        
    def index_with(self, vocab: Vocabulary):
        self.vocab = vocab

In [5]:
class LossMetric(Metric):
    def __init__(self):
        self.sum = 0.
        self.count = 0.
    
    def __call__(self, loss: torch.Tensor):
        self.sum += loss.detach().cpu()
        self.count += 1
    
    def get_metric(self, reset: bool = False):
        if self.count > 1e-12:
            loss =  float(self.sum / self.count)
        else:
            loss = 0.
        if reset:
            self.reset()
        return loss
    
    def reset(self):
        self.sum = 0.
        self.count = 0.

In [6]:
class StateTracker(Model):
    def __init__(self,
                 sentence_embedder: TextFieldEmbedder,
                 hidden_size: int,
                 vocab: Vocabulary,
                 general_act_size: int = len(GENERAL_ACTS) + 2,
                 intent_act_size: int = 1,
                 slot_act_size: int = len(SLOT_ACTS),
                 dropout: float = 0.1) -> None:
        super().__init__(vocab)
        self.sentence_embedder = sentence_embedder
        self.dropout = torch.nn.Dropout(dropout)
        emb_size = sentence_embedder.get_output_dim()
        self.intent_projection = torch.nn.Linear(emb_size + intent_act_size, hidden_size)
        self.slot_projection = torch.nn.Linear(emb_size + slot_act_size, hidden_size)
        self.gru = torch.nn.GRU(input_size=hidden_size * 2 + emb_size + general_act_size,
                                hidden_size=hidden_size)
        self.h_0 = torch.nn.Parameter(torch.randn(1, 1, hidden_size))
        self.intent_output_layer = torch.nn.Linear(hidden_size + emb_size*2 + intent_act_size + general_act_size, 1)
        self.none_and_unchanged_layer = torch.nn.Linear(hidden_size, 2)
        self.slot_output_layer = torch.nn.Linear(hidden_size + emb_size*2 + slot_act_size + general_act_size, 1)
        self.intent_accuracy = CategoricalAccuracy()
        self.slot_accuracy = BooleanAccuracy()
        self.intent_loss = LossMetric()
        self.slot_loss = LossMetric()
        
    
    def get_sentence_embedding(self, tensor_dict: Dict[str, torch.Tensor]):
        token_embeddings = self.dropout(self.sentence_embedder(tensor_dict))
        mask = (tensor_dict['tokens'] > 0).byte()  # mask for original bert
        sentence_embedding = util.get_final_encoder_states(token_embeddings, mask, bidirectional=True)
        return sentence_embedding
    
    
    def get_valid_description_embeddings(self, descriptions: torch.Tensor):
        index = torch.arange((descriptions[:, 0] > 0).sum(), device=descriptions.get_device())
        valid_descriptions = descriptions.index_select(0, index)
        embeddings = self.get_sentence_embedding({'tokens': valid_descriptions})
        return embeddings
    
    
    def get_desc_emb_and_mask(self, tensor_dict: Dict[str, torch.Tensor]):
        emb_list = [self.get_valid_description_embeddings(descriptions)
                    for descriptions in tensor_dict['tokens']]
        emb = pad_sequence(emb_list, batch_first=True)
        mask = (tensor_dict['tokens'][:, :, 0] > 0).byte()
        return emb, mask


    def forward(self,
                meta: Dict[str, str],
                user_utterance: Dict[str, torch.Tensor],
                intent_descriptions: Dict[str, torch.Tensor],
                slot_descriptions: Dict[str, torch.Tensor],
                general_acts: torch.Tensor,
                intent_acts: torch.Tensor,
                slot_acts: torch.Tensor,
                intent_label: torch.Tensor = None,
                slot_label: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        user_utterance_emb = self.get_sentence_embedding(user_utterance)
        # user_utterance_emb: (turn_size, emb_size)
        
        intent_emb, intent_mask = self.get_desc_emb_and_mask(intent_descriptions)
        intent_input = self.dropout(torch.cat([intent_emb, intent_acts.unsqueeze(-1)], dim=2))
        intent_encodings = F.relu(self.intent_projection(intent_input))
        intent_encoding = util.masked_mean(
            intent_encodings, intent_mask.unsqueeze(-1).expand(-1, -1, intent_encodings.size(2)), dim=1)

        slot_emb, slot_mask = self.get_desc_emb_and_mask(slot_descriptions)
        slot_input = self.dropout(torch.cat([slot_emb, slot_acts], dim=2))
        slot_encodings = F.relu(self.slot_projection(slot_input))
        slot_encoding = util.masked_mean(
            slot_encodings, slot_mask.unsqueeze(-1).expand(-1, -1, slot_encodings.size(2)), dim=1)

        general_encoding = torch.cat([user_utterance_emb, general_acts], dim=1)
        gru_input = torch.cat([general_encoding, intent_encoding, slot_encoding],
                              dim=1).unsqueeze(1)  # trick: batch dim 1 here
        states, _ = self.gru(gru_input, self.h_0)
        # states: (turn_size, 1, hidden_size)
        
        turn_input = torch.cat([states, general_encoding.unsqueeze(1)], dim=2)
        
        intent_num = intent_encodings.size(1)
        intent_state = torch.cat([intent_input, turn_input.expand(-1, intent_num, -1)], dim=2)
        intent_logits = self.intent_output_layer(intent_state).squeeze(2)
        intent_logits.masked_fill_((1 - intent_mask), float('-inf'))
        special_logits = self.none_and_unchanged_layer(states.squeeze(1))  # NONE and UNCHANGED intents
        intent_logits = torch.cat([special_logits, intent_logits], axis=1)
        
        slot_num = slot_encodings.size(1)
        slot_state = torch.cat([slot_input, turn_input.expand(-1, slot_num, -1)], dim=2)
        slot_sigmoids = F.sigmoid(self.slot_output_layer(slot_state).squeeze())
        
        output = dict(meta=meta, intent_logits=intent_logits, slot_sigmoids=slot_sigmoids)

        if intent_label is not None:
            output['intent_label'] = intent_label
            output['slot_label'] = slot_label
            self.intent_accuracy(intent_logits, intent_label)
            self.slot_accuracy((slot_sigmoids > 0.5).float(), slot_label, slot_mask.float())
            intent_loss = F.cross_entropy(intent_logits, intent_label.long())
            slot_loss = util.masked_mean(F.binary_cross_entropy(slot_sigmoids, slot_label, reduce=False),
                                         slot_mask, dim=1).mean()
            self.intent_loss(intent_loss)
            self.slot_loss(slot_loss)
            output['loss'] = intent_loss + slot_loss
        return output
    
    
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"intent_acc": self.intent_accuracy.get_metric(reset),
                "slot_acc": self.slot_accuracy.get_metric(reset),
                "intent_loss": self.intent_loss.get_metric(reset),
                "slot_loss": self.slot_loss.get_metric(reset)}

In [7]:
reader = DialogueDatasetReader()
train_dataset = reader.read('data/train')
validation_dataset = reader.read('data/dev')
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)
HIDDEN_DIM = 512
token_embedding = PretrainedBertEmbedder(bert_model, top_layer_only=True)
sentence_embedder = BasicTextFieldEmbedder(
    {"tokens": token_embedding}, 
    embedder_to_indexer_map={"tokens": ["tokens"]},
    allow_unmatched_keys=True
)

175780it [06:54, 423.91it/s]
26077it [01:01, 423.47it/s]
100%|██████████| 201857/201857 [00:06<00:00, 31343.31it/s]


In [8]:
model = StateTracker(sentence_embedder, HIDDEN_DIM, vocab)
if torch.cuda.is_available():
    cuda_device = CUDA
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
iterator = DialogueIterator()
iterator.index_with(vocab)
serialization_dir = f'models/tensorboard_{EXP}'
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=3,
                  num_epochs=1,
                  serialization_dir=serialization_dir,
                  summary_interval=200,
                  grad_norm=5,
                  cuda_device=cuda_device)
trainer.train()

intent_acc: 0.4781, slot_acc: 0.8531, intent_loss: 3.5848, slot_loss: 0.1377, loss: 3.7225 ||:   1%|          | 200/16142 [08:48<2:34:26,  1.72it/s]  

intent_acc: 0.5203, slot_acc: 0.8559, intent_loss: 3.0827, slot_loss: 0.1480, loss: 3.2307 ||:   2%|▏         | 400/16142 [11:04<2:54:28,  1.50it/s]

intent_acc: 0.5505, slot_acc: 0.8573, intent_loss: 2.7788, slot_loss: 0.1376, loss: 2.9164 ||:   4%|▎         | 600/16142 [13:25<3:51:44,  1.12it/s]

intent_acc: 0.5727, slot_acc: 0.8575, intent_loss: 2.6307, slot_loss: 0.1298, loss: 2.7605 ||:   5%|▍         | 800/16142 [15:28<3:11:01,  1.34it/s]

intent_acc: 0.5933, slot_acc: 0.8582, intent_loss: 2.4831, slot_loss: 0.1283, loss: 2.6114 ||:   6%|▌         | 1000/16142 [17:33<2:58:22,  1.41it/s]

...

intent_acc: 0.7721, slot_acc: 0.8608, intent_loss: 1.2823, slot_loss: 0.1088, loss: 1.3911 ||:  97%|█████████▋| 15600/16142 [3:23:37<10:18,  1.14s/it]

intent_acc: 0.7726, slot_acc: 0.8609, intent_loss: 1.2790, slot_loss: 0.1086, loss: 1.3876 ||:  98%|█████████▊| 15800/16142 [3:26:18<05:44,  1.01s/it]

intent_acc: 0.7729, slot_acc: 0.8609, intent_loss: 1.2771, slot_loss: 0.1086, loss: 1.3856 ||:  99%|█████████▉| 16000/16142 [3:28:43<02:18,  1.03it/s]

intent_acc: 0.7732, slot_acc: 0.8608, intent_loss: 1.2756, slot_loss: 0.1088, loss: 1.3845 ||: 100%|██████████| 16142/16142 [3:30:26<00:00,  1.34it/s]
intent_acc: 0.8809, slot_acc: 0.8820, intent_loss: 0.4040, slot_loss: 0.0607, loss: 0.4647 ||: 100%|██████████| 2482/2482 [28:33<00:00,  1.36it/s]


{'best_epoch': 0,
 'peak_cpu_memory_MB': 8097.164,
 'peak_gpu_0_memory_MB': 27349,
 'peak_gpu_1_memory_MB': 7416,
 'peak_gpu_2_memory_MB': 5558,
 'peak_gpu_3_memory_MB': 19977,
 'training_duration': '3:59:01.512742',
 'training_start_epoch': 0,
 'training_epochs': 0,
 'epoch': 0,
 'training_intent_acc': 0.773165320286722,
 'training_slot_acc': 0.8608487882580498,
 'training_intent_loss': 1.2756425142288208,
 'training_slot_loss': 0.1088167279958725,
 'training_loss': 1.3844616618133105,
 'training_cpu_memory_MB': 8097.164,
 'training_gpu_0_memory_MB': 27349,
 'training_gpu_1_memory_MB': 7416,
 'training_gpu_2_memory_MB': 5558,
 'training_gpu_3_memory_MB': 19977,
 'validation_intent_acc': 0.8809295547800744,
 'validation_slot_acc': 0.8819649499558998,
 'validation_intent_loss': 0.4039567708969116,
 'validation_slot_loss': 0.06074870750308037,
 'validation_loss': 0.46470570491879043,
 'best_validation_intent_acc': 0.8809295547800744,
 'best_validation_slot_acc': 0.8819649499558998,
 'bes

In [9]:
# save model
with open(f"/tmp/model_{EXP}.th", 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files(f"/tmp/vocabulary_{EXP}")

In [10]:
def predict(file_path, output_path=f'data/predict/{EXP}', slot_threshold=0.5):
    model.eval()
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    if file_path == 'data/dev':
        eval_dataset = validation_dataset
    else:
        eval_dataset = reader.read(file_path)
    result = {}
    all_outputs = []
    for inputs in tqdm(iterator(eval_dataset, num_epochs=1)):
        last_service = None
        last_active_intent = None
        inputs = util.move_to_device(inputs, cuda_device)
        outputs = model(**inputs)
        all_outputs.append(outputs)
        for meta, intent_logits, slot_sigmoids in zip(outputs['meta'], 
                                                      outputs['intent_logits'], 
                                                      outputs['slot_sigmoids']):
            dialogue_id = meta['dialogue_id']
            turn_id = meta['turn_id']
            service = meta['service']
            key = (dialogue_id, turn_id, service)
            if last_service != service:  # Force to predict some intent
                intent_logits[0] = float('-inf')
                intent_logits[1] = float('-inf')
            intent_num = len(reader.index_to_intent[service])
            intent_index = intent_logits[:intent_num].argmax()
            active_intent = reader.index_to_intent[service][intent_index]
            if active_intent == 'UNCHANGED':
                active_intent = last_active_intent
            else:
                last_active_intent = active_intent
            slot_num = len(reader.index_to_slot[service])
            requested_slots = [reader.index_to_slot[service][i] 
                               for i, sigmoid in enumerate(slot_sigmoids[:slot_num]) if sigmoid > slot_threshold]
            state = dict(active_intent=active_intent, requested_slots=requested_slots)
            result[key] = state
            last_service = service
    for file in sorted(glob.glob(os.path.join(file_path, 'dialogues*.json'))):
        with open(file, 'r') as f:
            ds = json.load(f)
        for dialog in ds:
            dialogue_id = dialog['dialogue_id']
            turn_id = 0
            last_service = None
            for turn in dialog['turns']:
                if turn['speaker'] == 'SYSTEM':  # system action encoding
                    turn_id += 1
                    continue
                if len(turn['frames']) > 1:  # sort
                    sorted_frames = []
                    for frame in turn['frames']:
                        if frame['service'] == last_service:
                            sorted_frames = [frame] + sorted_frames
                        else:
                            sorted_frames.append(frame)
                    turn['frames'] = sorted_frames
                for frame in turn['frames']:
                    service = frame['service']
                    key = (dialogue_id, turn_id, service)
                    state = result[key]
                    state['slot_values'] = frame['state']['slot_values']
                    frame['state'] = state
                last_service = turn['frames'][-1]['service']
        with open(os.path.join(output_path, os.path.split(file)[-1]), 'w') as f:
            json.dump(ds, f, indent=2)
    return all_outputs

In [11]:
all_outputs = predict('data/dev')

2482it [30:36,  1.46it/s]
